# eRADAR 

In [0]:
import numpy as np
import pandas as pd

### Load the patient data
- /Volumes/biomedicalinformatics_analytics/dev_lab_johnson/watch/pair_watch_final_2025.csv

In [0]:
def load_feat_data(year):
    pt_data = pd.read_csv(f"/Volumes/biomedicalinformatics_analytics/dev_lab_johnson/watch/pair_watch_final_{year}.csv", header=0, index_col=0)
    pt_data = pt_data[["age", "female", "dx_chf", "dx_cebv", "dx_diab", "dx_diab_c", "dx_cpd", "dx_hypothyoidism", "dx_rf", "dx_lymphoma", "dx_solidtumor", "dx_ra", "dx_weightloss", "dx_fluid", "dx_anemia", "dx_bipolar", "dx_depression", "dx_trauma", "dx_tobacco", "dx_atrial", "dx_gait", "low_bmi", "high_bmi", "hbp", "op_primary_care_visit", "ed_visit", "home_visit", "pt_visit", "cognitive_visit", "med_antidepressant", "med_sleepaid"]].astype(float)   # isolating and reordering features 
    return pt_data

In [0]:
data_2025 = load_feat_data(2025)
data_2024 = load_feat_data(2024)
data_2023 = load_feat_data(2023)

Prevalence of eRADAR features.

In [0]:
def feature_prevalence(data, summarize=False):
    ct = data.iloc[:, 1:].sum(axis=0)
    ct.name = "ct"

    pct = round(100 * data.iloc[:, 1:].sum(axis=0) / data.shape[0], 1)
    pct.name = "pct"

    prevalence = pd.merge(ct, pct, left_index=True, right_index=True)
    prevalence.loc["age"] = [data["age"].mean(), data["age"].std()]
    
    if summarize:
        prevalence = prevalence.apply(lambda x: f"{x["ct"]:.1f} ({x["pct"]:.1f})", axis=1)
    
    return prevalence

In [0]:
prev_2025 = feature_prevalence(data_2025, summarize=True)
prev_2025.name = 2025

prev_2024 = feature_prevalence(data_2024, summarize=True)
prev_2024.name = 2024

prev_2023 = feature_prevalence(data_2023, summarize=True)
prev_2023.name = 2023

prev = pd.merge(prev_2025, prev_2024, left_index=True, right_index=True)
prev = pd.merge(prev, prev_2023, left_index=True, right_index=True)
prev

### Apply the eRADAR model

In [0]:
class eRADAR:
    def __init__(self):
        self.intercept = -11.83
        self.weights = [0.11, -0.10, 0.28, 0.18, 0.06, 0.34, -0.11, 0.05, -0.15, 0.01, -0.17, -0.05, -0.34, -0.07, -0.25, 0.46, 0.03, 0.20, -0.15, 0.03, 0.24, 0.29, -0.11, -0.09, -0.55, 0.34, 0.00, -0.13, 0.47, 0.58, 0.03]

    def score(self, x):
        return 1 / (1 + np.exp(-self.intercept - np.dot(x, self.weights)))

In [0]:
model = eRADAR()

scores_2025 = data_2025.apply(model.score, axis=1)
scores_2024 = data_2024.apply(model.score, axis=1)
scores_2023 = data_2023.apply(model.score, axis=1)

### Analysis

In [0]:
import matplotlib.pyplot as plt

#### Recruitment pool size for varying eRADAR score cutoff percentiles.

In [0]:
def recruitment_summary(data, scores):
    qs = [99] + list(range(95, 45, -5))
    thresholds = np.percentile(scores, qs)
    q_thresh = dict(zip(qs, thresholds))

    pool_size   = [(scores >= t).sum() for t in thresholds]
    ages        = [(round(data.loc[(scores >= t), "age"].mean()), round(data.loc[(scores >= t), "age"].std(), 2)) for t in thresholds]
    females     = [round(100 * data.loc[(scores >= t), "female"].sum() / (scores >= t).sum(), 1) for t in thresholds]

    recruitment = pd.DataFrame({"Cutoff": np.round(100 * thresholds, 1), "Rec. Pool Size": pool_size, "Age (mean, std)": ages, "Pct. Female": females}, index=qs)
    recruitment.index.name = "Cutoff Percentile"
    return recruitment, q_thresh

In [0]:
rec_2025, q_thresh_2025 = recruitment_summary(data_2025, scores_2025)
rec_2024, q_thresh_2024 = recruitment_summary(data_2024, scores_2024)
rec_2023, q_thresh_2023 = recruitment_summary(data_2023, scores_2023)

rec = pd.merge(rec_2025, rec_2024, left_index=True, right_index=True, suffixes=("_2025", "_2024"))
rec = pd.merge(rec, rec_2023, left_index=True, right_index=True, suffixes=("", "_2023"))
rec

#### Distribution of eRADAR scores.

In [0]:
def plot_score_dist(scores, thresh_dict, year, bins=500, log_y_scale=True):
    fig, ax = plt.subplots(figsize=(12,5))
    ax.hist(scores, bins=bins, label=(year if type(scores) is tuple else None))

    if thresh_dict is not None:
        for q in [99, 95, 90, 80, 55]:
            ax.axvline(thresh_dict[q], color="red", linestyle="--", alpha=0.6)
            ax.text(thresh_dict[q], 1.02, f"{q}th", transform=plt.gca().get_xaxis_transform(), color="red", rotation=90, ha="center")

    ax.set_xlim([0, 1])
    ax.set_xticks(np.arange(0.0, 1.1, 0.1))
    ax.set_yscale("log" if log_y_scale else "linear")
    ax.set_xlabel("eRADAR Score")
    ax.set_ylabel("Frequency (Num. Patients)")
    ax.set_title(f"Distribution of eRADAR Scores for Penn {year} Patients", y=1.05)
    ax.grid()

    if type(scores) is tuple:
        ax.legend(loc="upper right")

    return fig, ax

In [0]:
fig, ax = plot_score_dist((scores_2025, scores_2024, scores_2023), None, (2025, 2024, 2023))
plt.show()
plt.close()

In [0]:
print("\tMedian [IQR]")
for (year, scores) in [(2025, scores_2025), (2024, scores_2024), (2023, scores_2023)]:
    print("%d\t%.1f%% [%.1f - %.1f%%]" % (year, 100 * np.median(scores), 100 * np.quantile(scores, 0.25), 100 * np.quantile(scores, 0.75)))

In [0]:
fig, ax = plot_score_dist(scores_2025, q_thresh_2025, 2025)
plt.show()
plt.close()

In [0]:
fig, ax = plot_score_dist(scores_2024, q_thresh_2024, 2024)
plt.show()
plt.close()

In [0]:
fig, ax = plot_score_dist(scores_2023, q_thresh_2023, 2023)
plt.show()
plt.close()

Distribution for REDUCE patient subset.

In [0]:
watch_mrns = pd.read_csv("/Volumes/biomedicalinformatics_analytics/dev_lab_johnson/watch/pair_watch_mrn_2025.csv", index_col=0)
reduce_mrns = pd.read_excel("/Volumes/biomedicalinformatics_analytics/dev_lab_johnson/watch/REDUCE_MRNs.xlsx")
reduce_mrns["MRN"] = reduce_mrns["MRN"].apply(lambda x: f"{x:09d}")

subset = watch_mrns.loc[watch_mrns["HUP_MRN"].isin(reduce_mrns["MRN"])].index.values

In [0]:
fig, ax = plot_score_dist(scores_2025.loc[subset], q_thresh_2025, 2025, bins=10, log_y_scale=False)
ax.set_title(f"Distribution of eRADAR Scores for REDUCE Patients", y=1.05)
plt.show()
plt.close()

In [0]:
# get list of missing participants
reduce_mrns.loc[~reduce_mrns["Provider ID"].isin(["PR007", "PR008"]) & ~reduce_mrns["MRN"].isin(subset), "MRN"].to_csv("MRN_missing.csv")

#### Most influential features for high eRADAR scores.

##### Prevalence of features

In [0]:
from scipy.stats import chi2_contingency, ttest_ind

In [0]:
q = 85

threshold = q_thresh_2025[q]

lo_prev = feature_prevalence(data_2025.loc[scores_2025 <  threshold])
hi_prev = feature_prevalence(data_2025.loc[scores_2025 >= threshold])

comparison = pd.merge(lo_prev, hi_prev, left_index=True, right_index=True, suffixes=("_lo", "_hi"))
comparison["Statistic"] = np.nan
comparison["p-value"] = np.nan
comparison["Odds Ratio"] = np.nan

for feature in comparison.index:
    if feature == "age":
        result = ttest_ind(data_2025.loc[scores_2025 >= threshold, "age"], data_2025.loc[scores_2025 < threshold, "age"])
        comparison.loc[feature, "Statistic"] = result.statistic
        comparison.loc[feature, "p-value"] = result.pvalue
    else:
        observed = np.array([
            [hi_prev.loc[feature, "ct"], (scores_2025 >= threshold).sum() - hi_prev.loc[feature, "ct"]],
            [lo_prev.loc[feature, "ct"], (scores_2025 <  threshold).sum() - lo_prev.loc[feature, "ct"]]
        ])

        result = chi2_contingency(observed)
        comparison.loc[feature, "Statistic"] = result.statistic
        comparison.loc[feature, "p-value"] = result.pvalue
        comparison.loc[feature, "Odds Ratio"] = (observed[0,0] * observed[1,1]) / (observed[0,1] * observed[1,0])

In [0]:
comparison["Low Risk"] = comparison.apply(lambda x: f"{x["ct_lo"]:.1f} ({x["pct_lo"]:.1f})", axis=1)
comparison["High Risk"] = comparison.apply(lambda x: f"{x["ct_hi"]:.1f} ({x["pct_hi"]:.1f})", axis=1)
comparison["Delta"] = comparison.apply(lambda x: abs(x["pct_lo"] - x["pct_hi"]), axis=1)
comparison.loc["age", "Delta"] = abs(comparison.loc["age", "ct_lo"] - comparison.loc["age", "ct_hi"])
comparison[["Low Risk", "High Risk", "Delta", "Statistic", "p-value", "Odds Ratio"]].sort_values(by=["p-value"])

##### SHAP values

In [0]:
import shap

In [0]:
q = 85

threshold = q_thresh_2025[q]

explainer = shap.Explainer(model.score, data_2025)
shap_values = explainer(data_2025)

In [0]:
shap.plots.beeswarm(shap_values[(scores_2025 >= threshold).values], max_display=data_2025.shape[1])

In [0]:
shap.plots.beeswarm(shap_values, max_display=data_2025.shape[1])